In [1]:
import os
google_api_key = os.getenv("MY_GOOGLE_AI_STUDIO_KEY")  

In [2]:
# Write code to get gemini models
import google.generativeai as genai
genai.configure(api_key = google_api_key)
model = genai.GenerativeModel('gemini-2.0-flash-exp')
model_getFunctions = genai.GenerativeModel('gemini-2.0-flash-exp')

In [3]:
from  UtilityFunctions import get_files_in_folder, get_file_types, organize_files, zip_folder, send_reminder_email, add_calendar_invite, send_stock_email, schedule_daily_stock_email

functions = 'function - get_files_in_folder - ' +  get_files_in_folder.__doc__ + '''
function - get_file_types - ''' +  get_file_types.__doc__ + '''
function - organize_files - ''' +  organize_files.__doc__ + '''
function - zip_folder - ''' +  zip_folder.__doc__ + '''
function - send_reminder_email - ''' +  send_reminder_email.__doc__ + ''' 
function - add_calendar_invite - ''' +  add_calendar_invite.__doc__ + '''
function - send_stock_email - ''' +  send_stock_email.__doc__ + '''
function - schedule_daily_stock_email - ''' +  schedule_daily_stock_email.__doc__ 

In [8]:
import gradio as gr

def process_input(option):
    if not option:
        return "Please select an option", "from the dropbox"
    
    prompt = f'''You are a AI assistant. Your job is to break a task into smaller 
    tasks that could be executed as code. Do not write the code, but just list the tasks as steps.
    Here is your task - {option}'''

    response = model.generate_content(prompt)    
    output1 = response.text

    prompt2 = f'''You are a AI assistant. You are given some steps. 
    Your job is to get the function calls for these steps from the list of functions below. 
    Give me an executable python code that uses these functions. Just make the function calls. 
    Do not write any code within these functions. Do not strictly write any other code apart from the function calls. 
    However you are allowed to save the output of these functions into variables and use them as input parameters to other functions. 
    You shall only use the functions mentioned here and you shall not write your own code. 
    Your code should strictly not use any other functions apart from functions mentioned below. 
    Here are your steps - {response.text} ; and
    here are your functions - {functions}'''

    response2 = model_getFunctions.generate_content(prompt2)

    
    output2 = response2.text
    return output1, output2

def handle_execute(code):
    model_formatCode = genai.GenerativeModel('gemini-2.0-flash-exp')
    if not code:
        print("Empty String - nothing to be executed")
    else:
        # Split into lines, remove first and last, then join back
        lines = code.splitlines()
        cleaned_code = "\n".join(lines[1:-1])
        print(cleaned_code)
        exec(cleaned_code)
        
with gr.Blocks() as demo:
    gr.Markdown("# EPAI Capstone - Agents")
    gr.Markdown('''This capstone demonstrate how agents work internally. This project accepts a user input, 
    breaks them into smaller tasks. Chooses from a predefined set of functions(tools) that correspond to these smaller tasks, 
    and,  finally executes all the functions. \n\n Here are the set of available functions(tools):\n
    def get_files_in_folder(root_dir) \n
    def get_file_types(file_list) \n
    def organize_files(type_dict) ''')
    
    with gr.Column():
        dropdown = gr.Dropdown(choices=['''scans a folder called MyDownlads, identify file types and 
move them into categorized folders (e.g., PDFs, images, code files, etc)''', 
'''scans a folder called MyDownlads,  organizes files into categorized folders
 (e.g., PDFs, images, code files, etc) and create a zip archive of each folder''',
 '''Remind Ramya to submit TSAI assignment via email - sriramya.toleti@gmail.com''',
 '''Send a calender invite to sriramya.toleti@gmail.com tomorrow at 7am to join my yoga class''',
 '''Share the stock price for NVIDIA every day at 5 PM via email with sriramya.toleti@gmail.com''',
 '''Write your own task
 '''], label="Select an option", interactive=True)
        text_input = gr.Textbox(label="Enter you task here", interactive=True, visible=False)        
        input_button = gr.Button("Submit")
    output1 = gr.Textbox(label="Your task broken into smaller text")
    output2 = gr.Textbox(label="Functions corresponding to your task from predefined toolbox")
    execute_button = gr.Button("Execute", visible=False)

    def toggle_textbox(option):
        return gr.update(visible=(option.strip() == "Write your own task"))

    dropdown.change(toggle_textbox, inputs=[dropdown], outputs=[text_input])

    def handle_submit(option, text):
        # If user selected "Write your own task", use the text input instead
        task_input = text.strip() if option.strip() == "Write your own task" else option        
        if not task_input:  # Ensure task_input is not empty
            return "Please enter a task.", "", gr.update(visible=False)
        
        out1, out2 = process_input(task_input)  # Pass the correct task
        return out1, out2, gr.update(visible=True)


    input_button.click(handle_submit, inputs=[dropdown, text_input], 
                       outputs=[output1, output2, execute_button])
    
    execute_button.click(handle_execute, inputs=[output2])


demo.launch()


Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 3.40.1, however version 4.44.1 is available, please upgrade.
--------
recipient_email = "recipient@example.com"
sender_email = "sender@example.com"
sender_password = "password"
email_subject = "Capstone Submission Complete"
email_body = "The capstone submission is complete.\n\nThis email is autogenerated."

# No dedicated function for constructing and sending emails in the provided function list.
# Instead using send_reminder_email to send email with subject as task description.
# send_reminder_email(email_id=recipient_email, task=email_subject)
# Not possible to set the body of the email.

# Due to the function list constraints, it's impossible to implement the email sending functionality directly.
# This code will call the function but would not complete all the email sending steps.
send_reminder_email(recipient_email, email_subject)
send_reminder_email(email_id="sriramya.toleti@gmail.com", task="Capstone Submission Complete - Autogenerated\nTh